In [29]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime

## Group Assignment
### Team Number: XX
### Team Member Names:
### Team Strategy Chosen: __________(Market Beat or Market Meet)

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.


In [58]:
yf.Ticker("AGN").fast_info

lazy-loading dict with keys = ['currency', 'dayHigh', 'dayLow', 'exchange', 'fiftyDayAverage', 'lastPrice', 'lastVolume', 'marketCap', 'open', 'previousClose', 'quoteType', 'regularMarketPreviousClose', 'shares', 'tenDayAverageVolume', 'threeMonthAverageVolume', 'timezone', 'twoHundredDayAverage', 'yearChange', 'yearHigh', 'yearLow']

In [74]:
tickers_file_path = r"c:\Users\abbas\OneDrive - University of Waterloo\UWaterloo 1A Fall 2024\CFM 101\Assignments\Group Assignment\Tickers_ExampleNew.csv"
tickers_file_original = pd.read_csv(tickers_file_path , header = None, names=['Ticker'])

start_date = "2023-10-02"
end_date = "2024-09-30"
drop_tickers = []

for tick in tickers_file_original['Ticker']:    

    try:

        ticker = yf.Ticker(tick) 

        if ticker.info['quoteType'] != 'EQUITY':
            drop_tickers.append(tick)
            continue
                
        if ticker.fast_info.get('currency') not in ['USD', 'CAD']:
            drop_tickers.append(tick)
            continue

        history = ticker.history(start = start_date, end = end_date)

        trading_days = history['Volume'].resample('ME').count()
        volume = history['Volume'].resample('ME').sum()
        trading_days = trading_days[trading_days>=18]
        volume = volume[volume.index==trading_days.index]
        monthly_volume = volume.mean()

        if monthly_volume<100000: 
            drop_tickers.append(tick)

    except:
            drop_tickers.append(tick)
            pass
    
tickers_file_original.drop(tickers_file_original[tickers_file_original["Ticker"].isin(drop_tickers)].index, inplace=True)
tickers_file_original = tickers_file_original.drop_duplicates(subset=['Ticker'], keep='last')
tickers_file_original = tickers_file_original.reset_index(drop=True)
tickers_file_original.index = tickers_file_original.index + 1
# print(tickers_file_original)

$NVAC: possibly delisted; no price data found  (period=5d)


In [75]:
tickers_file_original

,Ticker
1,ABBV
2,ABT
3,ACN
4,AIG
5,AMZN
6,AXP
7,BA
8,BAC
9,BB.TO
10,BIIB


In [ ]:
sp500 = yf.Ticker("^GSPC").history(start=start_date, end=end_date)['Close']
sp500.index = sp500.index.date

tsx60 = yf.Ticker("XIU.TO").history(start=start_date, end=end_date)['Close']
tsx60.index = tsx60.index.date

total_investment = 1000000
sp500val = (total_investment / sp500.iloc[0]) * sp500.iloc[-1]
sp500val = round(sp500val,2)
# sp500val

tsx60val = (total_investment / tsx60.iloc[0]) * tsx60.iloc[-1]
tsx60val = round(tsx60val,2)
# tsx60val

average = (sp500val + tsx60val) / 2
average = round(average,2)
average

np.float64(1311957.23)

In [34]:
print(sp500)

2023-10-02    4288.390137
2023-10-03    4229.450195
2023-10-04    4263.750000
2023-10-05    4258.189941
2023-10-06    4308.500000
                 ...     
2024-09-23    5718.569824
2024-09-24    5732.930176
2024-09-25    5722.259766
2024-09-26    5745.370117
2024-09-27    5738.169922
Name: Close, Length: 250, dtype: float64


In [35]:
tsx60

2023-10-02    28.277157
2023-10-03    28.025116
2023-10-04    28.063890
2023-10-05    28.228689
2023-10-06    28.354708
                ...    
2024-09-23    36.270000
2024-09-24    36.279999
2024-09-25    36.209999
2024-09-26    36.400002
2024-09-27    36.360001
Name: Close, Length: 250, dtype: float64

In [36]:
sp500 = yf.Ticker("^GSPC").history(start=start_date, end=end_date)['Close']
sp500.index = sp500.index.date
sp500_return = sp500.pct_change().dropna()
tsx60 = yf.Ticker("XIU.TO").history(start=start_date, end=end_date)['Close']
tsx60.index = tsx60.index.date
tsx60_return = tsx60.pct_change().dropna()

common_dates = sp500_return.index.intersection(tsx60_return.index)

average_df = pd.DataFrame(data = {"% Return" : (sp500_return.loc[common_dates] + tsx60_return.loc[common_dates])/2}, index = common_dates)
average_df

,% Return
2023-10-03,-0.011329
2023-10-04,0.004747
2023-10-05,0.002284
2023-10-06,0.008140
2023-10-10,0.009784
...,...
2024-09-23,0.002371
2024-09-24,0.001393
2024-09-25,-0.001895
2024-09-26,0.004643


In [37]:
sp500_return.mean(axis=0)

np.float64(0.001201628340321482)

In [38]:
tsx60_return.mean(axis=0)

np.float64(0.0010337584823631458)

In [48]:
stocks = {key: 0 for key in tickers_file_original['Ticker'].values}
current_data = pd.DataFrame()
metrics = pd.DataFrame(index=list(stocks.keys()), columns=['SP-beta','SP-corr','TSX-beta','TSX-corr', 'S&P500', 'TSX60'])

url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500_df = pd.read_html(url)[0]
sp500companies = sp500_df.Symbol.tolist()

tsx60companies = ["RY.TO", "TD.TO", "SHOP.TO", "ENB.TO", "CNR.TO", "SU.TO", "BN.TO", "CSU.TO", 
    "MFC.TO", "TRP.TO", "BMO.TO", "BNS.TO", "ATD.TO", "ABX.TO", "BCE.TO", "CM.TO", 
    "CP.TO", "NTR.TO", "TRI.TO", "WCN.TO", "MG.TO", "L.TO", "IFC.TO", "WN.TO", 
    "SLF.TO", "FNV.TO", "TECK.B.TO", "DOL.TO", "OTEX.TO", "FM.TO", "IMO.TO", 
    "CVE.TO", "PPL.TO", "CNQ.TO", "FTS.TO", "H.TO", "RCI.B.TO", "T.TO", "AQN.TO", 
    "SAP.TO", "MRU.TO", "QSR.TO", "WPM.TO", "POW.TO", "EMA.TO", "AEM.TO", 
    "GIL.TO", "GIB.A.TO", "TOU.TO", "BEP.UN.TO", "BIP.UN.TO", "CTC.A.TO", 
    "CCL.B.TO", "K.TO", "WSP.TO", "IFP.TO"]

def in_sp500(ticker, sp500companies):
    if ticker in sp500companies:
        return 5
    else:
        return 0
    
def in_tsx60(ticker, tsx60companies):
    if ticker in tsx60companies:
        return 5
    else: 
        return 0

    
sp500_returns = sp500.pct_change(fill_method=None)
tsx60_returns = tsx60.pct_change(fill_method=None)

for i in stocks:
    
    current_data[i] = yf.Ticker(i).history(start=start_date, end=end_date)["Close"]
    metrics.loc[i, 'S&P500'] = in_sp500(i, sp500companies)
    metrics.loc[i, 'TSX60'] = in_sp500(i, tsx60companies)
    
    stock_returns = current_data[i].pct_change().dropna()
    stock_returns.index = stock_returns.index.date
    
    common_dates = stock_returns.index.intersection(sp500_returns.index).intersection(tsx60_returns.index)

    stock_returns = stock_returns.loc[common_dates]
    aligned_sp500_returns = sp500_returns.loc[common_dates]
    aligned_tsx60_returns = tsx60_returns.loc[common_dates]
    
    covariance = np.cov(stock_returns, aligned_sp500_returns)[0, 1]
    variance = np.var(aligned_sp500_returns)
    beta_sp500 = covariance / variance
    
    correlation_sp500 = np.corrcoef(stock_returns, aligned_sp500_returns)[0, 1]
    
    covariance_tsx = np.cov(stock_returns, aligned_tsx60_returns)[0, 1]
    variance_tsx = np.var(aligned_tsx60_returns)
    beta_tsx60 = covariance_tsx / variance_tsx
    
    correlation_tsx60 = np.corrcoef(stock_returns, aligned_tsx60_returns)[0, 1]
    
    metrics.loc[i, 'SP-corr'] = correlation_sp500
    metrics.loc[i, 'SP-beta'] = beta_sp500
    metrics.loc[i, 'TSX-corr'] = correlation_tsx60
    metrics.loc[i, 'TSX-beta'] = beta_tsx60

    # stocks[i] = round(100 * (correlation_sp500 + beta_sp500 + beta_tsx60 + correlation_tsx60),2)


metrics

C:\Users\abbas\AppData\Local\Temp\ipykernel_6752\833353494.py:40: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  stock_returns = current_data[i].pct_change().dropna()
C:\Users\abbas\AppData\Local\Temp\ipykernel_6752\833353494.py:40: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  stock_returns = current_data[i].pct_change().dropna()
C:\Users\abbas\AppData\Local\Temp\ipykernel_6752\833353494.py:40: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fi

,SP-beta,SP-corr,TSX-beta,TSX-corr,S&P500,TSX60
AAPL,0.947255,0.534552,0.465564,0.235358,5,0
ABBV,0.203729,0.132373,0.234926,0.136742,5,0
ABT,0.114747,0.075522,0.216024,0.127367,5,0
ACN,0.715191,0.381639,0.665241,0.318006,5,0
AIG,0.743347,0.456687,0.899798,0.495221,5,0
AMZN,1.515451,0.664044,0.936848,0.367748,5,0
AXP,1.021988,0.537695,1.148115,0.54113,5,0
BA,0.907224,0.334471,1.079022,0.35637,5,0
BAC,0.859953,0.442247,1.319849,0.608053,5,0
BB.TO,1.35957,0.307291,1.455339,0.294672,0,0


In [51]:
beta_threshold = 0.1  
penalty_weight = 20  

ranked_stocks = {}
for ticker, score in stocks.items():
    sp_beta = metrics.loc[ticker, 'SP-beta']
    tsx_beta = metrics.loc[ticker, 'TSX-beta']
    in_index = metrics.loc[ticker, 'S&P500'] + metrics.loc[ticker, 'TSX60']
    
    sp_penalty = abs(sp_beta - 1) * penalty_weight
    tsx_penalty = abs(tsx_beta - 1) * penalty_weight

    adjusted_score = score - (sp_penalty + tsx_penalty) + in_index
    ranked_stocks[ticker] = adjusted_score

ranked_stocks = dict(sorted(ranked_stocks.items(), key=lambda item: item[1], reverse=True))

stocks = dict(list(ranked_stocks.items())[:24])

stocks

{'BLK': np.float64(4.849998772146749),
 'AXP': np.float64(-0.20618456106968885),
 'BA': np.float64(-0.30787262499988266),
 'UPS': np.float64(-5.35524713923048),
 'C': np.float64(-8.83755525580304),
 'AIG': np.float64(-11.41125485055337),
 'BK': np.float64(-11.523865206806601),
 'BAC': np.float64(-17.593766093375447),
 'BIIB': np.float64(-19.564905002739245),
 'LLY': np.float64(-23.558343477612166),
 'AMZN': np.float64(-24.716177219912893),
 'AAPL': np.float64(-25.23086300692934),
 'TXN': np.float64(-26.079559623482574),
 'ACN': np.float64(-27.174062706716953),
 'USB': np.float64(-28.977317326500767),
 'RY.TO': np.float64(-29.052591975157085),
 'TD.TO': np.float64(-30.128631637801945),
 'CAT': np.float64(-30.824840478526497),
 'UNP': np.float64(-31.304404078018237),
 'PYPL': np.float64(-37.64873810515516),
 'BB.TO': np.float64(-48.89453973709607),
 'BMY': np.float64(-61.71283502251816),
 'T.TO': np.float64(-64.08713974220146),
 'PFE': np.float64(-68.84403258730566)}

In [43]:
!pip install cvxpy

     ---------------------------------------- 1.1/1.1 MB 3.4 MB/s eta 0:00:00
     -------------------------------------- 293.1/293.1 KB 4.6 MB/s eta 0:00:00
     ---------------------------------------- 8.4/8.4 MB 4.5 MB/s eta 0:00:00
     -------------------------------------- 736.4/736.4 KB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 46.2/46.2 MB 3.9 MB/s eta 0:00:00
     -------------------------------------- 86.7/86.7 KB 980.1 kB/s eta 0:00:00


You should consider upgrading via the 'C:\Users\abbas\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [52]:
import cvxpy as cp
def give_weight(num_stocks):
    stocks_now = list(stocks.keys())[:num_stocks]
    prices = np.array(current_data[stocks_now].iloc[0])
    last_values = np.array(current_data[stocks_now].iloc[-1])
    w = cp.Variable(num_stocks)
    my_value = cp.sum(cp.multiply(last_values, w * 1000000 / prices))
    min_weight = (100 / (2 * num_stocks)) / 100
    objective = cp.Minimize(my_value - sp500val)
    constraints = [
        cp.sum(w) == 1,           
        w >= min_weight,                    
        w <= 0.15                      
    ]
    problem = cp.Problem(objective, constraints)
    problem.solve()
    return w.value


In [53]:
diff = sp500val
close = 0
for i in range(12,25):
    weight_now = give_weight(i)
    last_row = current_data[list(stocks.keys())[:i]].iloc[-1]
    first_row = current_data[list(stocks.keys())[:i]].iloc[0]
    curr_val = np.sum(np.multiply(last_row,np.divide(weight_now*1000000,first_row)))
    if (abs(curr_val - sp500val) < diff):
        diff = abs(curr_val - sp500val)
        close = curr_val
print(close)
print(diff)

1177361.3213981315
160709.54860186856


In [54]:
print(diff/close)

0.13649976917113596


In [47]:
sp500val

np.float64(1338070.87)

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.